Create Material for layered QW Object

In [44]:
from solcore import si, material
from solcore.structure import Layer, Structure
import solcore.quantum_mechanics as QM
import matplotlib.pyplot as plt
from solcore.structure import TunnelJunction
from solcore.solar_cell_solver import default_options
from solcore.analytic_solar_cells import parametric_tunnel_junction
import solcore.poisson_drift_diffusion as PDD
import numpy as np


# First we create the materials we need
top = material("GaAs")(T=293, strained = False, Na=8e22)
barrier = material("GaAsP")(T=293, P = 0.32, strained = False, Nd=1e24)
well = material("InGaAs")(T=293, In = 0.11, strained = False, Nd=1e24)
bottom = material("AlGaAs")(T=293, Al = 0.5, Strained = False, Nd=1e24) #doesn't recognize name pf compound

# As well as some of the layers
top_layer = Layer(width=si("10nm"), material=top)
barrier_layer = Layer(width=si("2.5nm"), material=barrier, role = 'barrier')
well_layer = Layer(width=si("8.5nm"), material=well, role = 'well')
bottom_barrier_layer = Layer(width=si("50nm"), material=bottom)
bottom_layer = Layer(width=si("50nm"), material=top)


# Layered quantum wells
layers = [top_layer, barrier_layer] + 10 * [well_layer, barrier_layer] + [bottom_barrier_layer, bottom_layer]

Create QW Layered Object

In [45]:
m_e = 9.1093837015*10**(-31) #kg
h_bar = 1.054571817E-34#in J*s
epsilon0 = 8.854187817 * 10**(-12) #C**2/(N*m**2)
epsilon = 11.48*epsilon0
effective_mass = 0.1*m_e
doping_concentration = 10**25 #m^-3
prefactor = (4/(3*h_bar))*((effective_mass*epsilon)/doping_concentration)**0.5

tunnel = TunnelJunction(v_peak=0.2, j_peak=400, v_valley=0.9, j_valley=10, prefactor=prefactor, j01=1e-21, kind='parametric')
parametric_tunnel_junction(tunnel, default_options)

v = tunnel.voltage

In [46]:
T = 300
interlayers = [barrier_layer, well_layer, barrier_layer]
wavelengths = np.linspace(350, 1050, 1001) * 1e-9

QW = PDD.QWunit(interlayers, T=T, repeat=10, substrate=top)

# We solve the quantum properties of the QW, leaving the default values of all parameters
QW_list = QW.GetEffectiveQW(wavelengths=wavelengths)

Solving QW properties...


In [47]:
from solcore.structure import Junction

T = 300

## Materials for the BOTTOM junction
window_bottom = material('GaInP')(T=T, Nd=5e24, In=0.49)
n_GaAs = material('GaAs')(T=T, Nd=1e24)
p_GaAs = material('GaAs')(T=T, Na=8e22)
bsf_bottom = material('GaInP')(T=T, Na=5e24, In=0.49)

GaAs_junction = Junction([Layer(width=60e-9, material=window_bottom, role="Window"),
                          Layer(width=1000e-9, material=n_GaAs, role="Emitter")] +
                         QW_list +
                         [Layer(width=2000e-9, material=p_GaAs, role="Base"),
                          Layer(width=200e-9, material=bsf_bottom, role="BSF")], sn=1e6, sp=1e6, T=T, kind='PDD')

## Materials for the TOP junction
window_top = material('AlInP')(T=T, Nd=5e23, Al=0.53)
n_GaInP = material('GaInP')(T=T, Nd=5e23, In=0.49)
p_GaInP = material('GaInP')(T=T, Na=8e22, In=0.49)
bsf_top = material('AlInP')(T=T, Na=5e24, Al=0.53)

GaInP_junction = Junction([Layer(width=30e-9, material=window_top, role="window"),
                         Layer(width=100e-9, material=n_GaInP, role="Emitter"),
                           Layer(width=2000e-9, material=p_GaInP, role="Base"),
                           Layer(width=100e-9, material=bsf_top, role="BSF")], sn=1e3, sp=1e3, T=T, kind='PDD')

Define Solar Cell

In [48]:
# antireflective coating
MgF2 = material('MgF2')()
ZnS = material('ZnScub')()

Arc1 = Layer(width=110e-9, material=MgF2, role="ARC1")
Arc2 = Layer(width=60e-9, material=ZnS, role="ARC2")

from solcore.solar_cell import SolarCell

my_solar_cell = SolarCell([Arc1,
                           Arc2,
                           GaInP_junction,
                           tunnel,
                           GaAs_junction],
                          T=T, substrate=n_GaAs)

In [49]:
from solcore.solar_cell_solver import solar_cell_solver
from solcore.light_source import LightSource

light_source = LightSource(source_type='standard', version='AM1.5g', x=wavelengths,
                           output_units='photon_flux_per_m', concentration=1)

solar_cell_solver(my_solar_cell, 'qe',
                  user_options={'light_source': light_source, 'wavelength': wavelengths, 'optics_method': 'TMM'})

Calculating RAT...
Calculating absorption profile...
Solving QE of the solar cell...
Solving quantum efficiency...
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

...done!

Solving quantum efficiency...
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

...done!



In [50]:
from plotly.express import scatter, line
from pandas import DataFrame

df = {'efficiency': my_solar_cell.absorbed, 'wavelength': wavelengths}
line(df, y='efficiency', x='wavelength')

Plot IV Curve

In [51]:
con = np.logspace(0, 3, 19)
vint = np.linspace(-3.5, 4, 600)
V = np.linspace(-3.5, 0, 300)

solar_cell_solver(my_solar_cell, 'iv',
                  user_options={'light_source': light_source, 'wavelength': wavelengths, 'optics_method': None,
                                'light_iv': True, 'mpp': True, 'voltages': V, 'internal_voltages': vint})
IV_data = my_solar_cell.iv

Solving IV of the junctions...
Solving IV...
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Error: Material AlInP not in the database for the mobility. Reverting to GaAs.
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

...done!

Solving IV...
Processing structure...
...done!

Solving equilibrium...
...done!

...done!

Processing structure...
...done!

Solving equilibrium...
...done!

.

In [52]:
IV_data

State([('IV',
        (array([-3.5       , -3.48829431, -3.47658863, -3.46488294, -3.45317726,
                -3.44147157, -3.42976589, -3.4180602 , -3.40635452, -3.39464883,
                -3.38294314, -3.37123746, -3.35953177, -3.34782609, -3.3361204 ,
                -3.32441472, -3.31270903, -3.30100334, -3.28929766, -3.27759197,
                -3.26588629, -3.2541806 , -3.24247492, -3.23076923, -3.21906355,
                -3.20735786, -3.19565217, -3.18394649, -3.1722408 , -3.16053512,
                -3.14882943, -3.13712375, -3.12541806, -3.11371237, -3.10200669,
                -3.090301  , -3.07859532, -3.06688963, -3.05518395, -3.04347826,
                -3.03177258, -3.02006689, -3.0083612 , -2.99665552, -2.98494983,
                -2.97324415, -2.96153846, -2.94983278, -2.93812709, -2.9264214 ,
                -2.91471572, -2.90301003, -2.89130435, -2.87959866, -2.86789298,
                -2.85618729, -2.84448161, -2.83277592, -2.82107023, -2.80936455,
              

In [53]:
current = -1*IV_data['IV'][1]
voltage = -1*IV_data['IV'][0]

IV_df = DataFrame({'Current': current, 'Voltage': voltage})
line(IV_df, x = 'Voltage', y = 'Current')

In [32]:
my_solar_cell

 [],